In [2]:
!pip install transformers datasets tokenizers seqeval

In [3]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification


c:\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\miniconda3\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnin

In [10]:
from datasets import load_dataset

dataset = load_dataset("lhoestq/conll2003")

c:\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Arnav's Lappy\.cache\huggingface\hub\datasets--lhoestq--conll2003. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For bet

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [14]:
dataset.shape

{'train': (14041, 5), 'validation': (3250, 5), 'test': (3453, 5)}

In [15]:
dataset["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [17]:
dataset["train"].features["ner_tags"]

List(Value('int64'))

In [18]:
dataset["train"].description

''

In [20]:
!pip install snowflake-snowpark-python

  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached pyopenssl-25.3.0-py3-none-any.whl.metadata (17 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------  1.8/1.9 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 8.5 MB/s eta 0:00:00
Using cached tzlocal-5.3.1-py3-none-any.whl (18 kB)
Using cached asn1crypto-1.5.1-py2.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.22.1 requires pyarrow<20,>=4.0.0, but you have pyarrow 22.0.0 which is incompatible.


In [21]:
from snowflake.snowpark.session import Session
from dotenv import load_dotenv
import os
import pandas as pd

In [22]:
load_dotenv()
pd.set_option("display.max_columns", 20)
connection_params = {
    "account": os.getenv("SF_ACCOUNT"),
    "user": os.getenv("SF_USER"),
    "password": os.getenv("SF_PASSWORD"),
    "role": os.getenv("SF_ROLE"),
    "warehouse": os.getenv("SF_WAREHOUSE"),
    "database": os.getenv("SF_DATABASE"),
    "schema": os.getenv("SF_SCHEMA")
}

In [23]:
session = Session.builder.configs(connection_params).create()

news_snowpark_df = session.table("NEWS_ARTICLES")
news_df = news_snowpark_df.to_pandas()

In [32]:
news_df.columns

Index(['author', 'title', 'description', 'url', 'urlToImage', 'publishedAt',
       'content', 'source.id', 'source.name'],
      dtype='object')

In [49]:
print(news_df.loc[67, "content"])

NOBA Bank Group (OM:NOBA) shares have held steady over the past week, with little change in trading activity. Investors are watching for upcoming catalysts or financial updates that could shape senti… [+3312 chars]


In [50]:
entity_dict = {
    "Startup Name": 'STARTUP',
    "Founder Name": "FOUNDER",
    "Investment Company": "INVESTOR",
    "Funding Amount": "INVESTMENT",
    "Revenue": "ARR",
    "Percent change in revenue": "PERCHNG"
}

In [51]:
!pip install worldnewsapi

  Using cached worldnewsapi-2.2.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
Using cached worldnewsapi-2.2.0-py3-none-any.whl (51 kB)
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.22.1 requires pyarrow<20,>=4.0.0, but you have pyarrow 22.0.0 which is incompatible.


In [2]:
import worldnewsapi
from worldnewsapi.rest import ApiException


In [3]:
configuration = worldnewsapi.Configuration(
    host = "https://api.worldnewsapi.com"
)

configuration.api_key['apiKey'] = "402302269a2e4e90a029481f80f40a1d"
configuration.api_key['headerApiKey'] = "402302269a2e4e90a029481f80f40a1d"
with worldnewsapi.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = worldnewsapi.NewsApi(api_client)
    text = "Top Startups OR Top Startup Founders OR Startup funding OR Startup revenue OR VC funding startup"
    language = "en"
    try:
        # Search News
        api_response = api_instance.search_news(text=text, language=language)

        print("The response of NewsApi->search_news:\n")
        news_articles_dict= api_response.to_dict()
        print(news_articles_dict.keys())

    except Exception as e:
        print("Exception when calling NewsApi->search_news: %s\n" % e)




The response of NewsApi->search_news:

dict_keys(['offset', 'number', 'available', 'news'])


In [4]:
import pandas as pd
df = pd.json_normalize(news_articles_dict["news"])

df.columns

Index(['summary', 'image', 'sentiment', 'language', 'title', 'url',
       'source_country', 'id', 'text', 'publish_date', 'video', 'category',
       'authors'],
      dtype='object')

In [5]:
df["content"] = df.apply(lambda x: x["title"]+"\n"+x["text"], axis=1)

In [103]:
entity_dict["Buying"] = "BUY"
entity_dict

{'Startup Name': 'STARTUP',
 'Founder Name': 'FOUNDER',
 'Investment Company': 'INVESTOR',
 'Funding Amount': 'INVESTMENT',
 'Revenue': 'ARR',
 'Percent change in revenue': 'PERCHNG',
 'Buying': 'BUY'}

In [6]:
text = df["content"][0]

In [7]:
import re

for m in re.compile("Meta").finditer(text):
    print(m.group(), m.start(), m.end())

Meta 764 768
Meta 829 833
Meta 882 886
Meta 1131 1135
Meta 1244 1248
Meta 1936 1940


In [17]:
article_text = '''Intel pursued deals that boosted CEO Lip-Bu Tan&#39;s fortune, sources say SAN FRANCISCO, Dec 10 : When the chairman of AI chip startup',
 'Rivos wanted Intel to bid for the company, he had no need to phone the chip giant. That’s because the chairman of Rivos was also Intel’s',
 'CEO: Lip-Bu Tan.  Tan had pitched Intel’s board on buying Rivos in the summer of 2025, but he had no luck. The board told Tan he had a',
 'conflict in representing both Rivos’ interests and Intel’s, and he lacked a strategy on artificial intelligence to justify a deal, three',
 'people familiar with the events told Reuters.  Tan asked one of his lieutenants at Intel to pitch a new AI plan, leading to partnership',
 'talks with Rivos, the people said. But now there was a problem: social media giant Meta had been stalking Rivos and made an offer for the',
 'company.  Meta’s interest spurred Intel to make its own offer. Meta countered with a sweetened bid. The competition for the startup drove',
 'the deal and incentives above the $2 billion valuation that Rivos had sought in fundraising earlier this year. Some of the sources pinned',
 'this package at around $4 billion.  Meta announced plans to buy Rivos in September. By then the bidding process had boosted the startup’s',
 'returns at Meta’s expense.  Reuters was unable to determine how much the Intel CEO profited personally as a Rivos shareholder because the',
 'financials are not public. But in a blog post on its website, Tan’s venture-capital firm, Walden Catalyst, touted how he had delivered a',
 '“successful outcome” for its investors and congratulated the Rivos team for their “remarkable achievement.”  The events show one of at least',
 'three instances where Intel has pursued deals that benefit Tan financially either by exploring bids for startups or investing in them',
 'directly through Intel’s investment arm, Intel Capital, said two of the sources.   Intel declined to make Tan available for an interview for',
 'this story. Meta did not respond to requests for comment, and Rivos declined to comment.  VENTURE CAPITALIST AS CEO  Intel hired Tan in',
 'March in part for his experience as a venture capitalist and unparalleled industry connections as a longtime investor in tech companies.',
 'Those connections have helped Intel clinch a $5 billion investment from Nvidia and a $2 billion investment from SoftBank.  Since Tan’s',
 'arrival, Intel has implemented policies requiring Tan to recuse himself from participating in investment decisions where he might benefit,',
 'two sources said. Specifically, Tan cannot attend or vote in decision meetings of Intel’s board or Intel Capital’s investment committee if',
 'he has a conflict in a venture or company-wide transaction, the sources said.  Such recusals are commonplace in industry. But they had not',
 'been an issue to the same degree at Intel because its leadership prior to Tan had fewer potentially conflicting investments, three of the',
 'sources said.  In the event of a Tan recusal for Intel Capital, the first two sources said, decision-making authority for the venture unit’s',
 'investment committee goes to Chief Financial Officer David Zinsner, who reports to Tan.  Intel declined to make Zinsner available for an',
 'interview.  Intel’s board knew when it appointed Tan that his web of investments in chip and technology companies could create conflicts,',
 'but the board accepted this, hoping that Tan can revive the iconic U.S. chipmaker, which lost $19 billion last year, one of the people said.',
 'Intel’s 11 independent board directors did not individually comment on Tan, but in response to Reuters queries, an Intel spokesperson said:',
 '“The Board of Directors believes it’s important that Intel fully leverage his vast network and position Intel to capture the next wave of',
 'industry innovation and opportunity.”  Tan’s dealmaking comes as the administration of President Donald Trump agreed to make an $8.9 billion',
 'investment for what would be the largest ownership stake in the chipmaker, designating it as strategic to the U.S. and effectively making',
 'its citizens shareholders.  Some chip-industry analysts have said in research notes that they welcomed Tan’s industry relationships. “He has',
 'a wide view across the ecosystem,” said Bernstein analyst Stacy Rasgon in an interview with Reuters. “And that’s helpful for Intel.”  Tan',
 'does not perceive his dealmaking at Intel to be conflicted, said two of the sources, who are familiar with his thinking. Tan believes that',
 'his roles at these startups and at Intel make him uniquely able to negotiate transactions that benefit all parties, the people said.  In a',
 'statement, an Intel spokesperson disputed that Tan’s dealmaking posed any problem. “The company has an unwavering commitment to the highest',
 'standards of corporate governance, integrity, and accountability,” the spokesperson said. Tan’s “extensive relationships across the global',
 'semiconductor ecosystem are invaluable as Intel positions itself to capitalize on a rapidly evolving industry landscape.”  The Securities',
 'and Exchange Commission would not require Intel to disclose related-party transactions that could involve Tan until the spring of 2026, a',
 'year after its last disclosure, which took place the week after his start date.  U.S. regulations require such disclosures when transactions',
 'personally benefiting corporate officers exceed $120,000, though experts have said smaller dollar amounts can be material to investors if',
 'the information would inform a decision to sell or purchase stock.  DEALS TO REVIVE INTEL’S AI STRATEGY  Tan believed Intel needed to buy',
 'Rivos because earlier in-house efforts to enter the AI chip market had failed, one of the sources said. In the statement, the Intel',
 'spokesperson said Tan was “advancing its AI strategy” and “revitalizing its engineering-centric, customer-first culture.”  Rivos was one of',
 'the major targets for Intel where Tan had interests on both sides of the deal. Tan also pitched Intel’s board on buying the troubled AI',
 'computing startup SambaNova, where Tan served as executive chairman, the first three sources said.  The rationale - debated inside Intel -',
 'was that SambaNova also could provide more tech and talent to build AI chips, the sources said.  SambaNova declined to comment on any talks',
 'between the startup and Intel. “While we’re always exploring strategic options, our focus remains on accelerating the roadmap, delivering',
 'products to market, and supporting our customers,” a spokesperson said.  Intel also declined to comment on the SambaNova talks.  Tan’s',
 'portfolio has drawn scrutiny before. In April, Reuters documented how Tan’s investment firms had stakes in more than 600 Chinese companies,',
 'some with military ties, drawing a rebuke by Trump that the U.S. chip manufacturer’s CEO was "highly CONFLICTED.”  According to a White',
 'House official, Tan subsequently cleared up Trump’s concerns in an Oval Office meeting, which paved the way for collaboration on U.S.',
 'national and economic security. In September, Trump publicly celebrated Intel’s rising share price following Tan’s Nvidia deal.  Intel’s',
 'share price has roughly doubled since Tan’s appointment, outpacing the percentage gains of the S&P 500 and chip leader Nvidia in that time.',
 'TAKING CONTROL OF INTEL CAPITAL  Soon after his appointment as CEO, Tan took direct control of Intel’s investment arm, Intel Capital,',
 'reversing a plan to spin it off. Instead, he reorganized the company so that Intel Capital would report to him, according to two of the',
 'sources, for reasons Intel has not disclosed. Its investment committee would be composed solely of Tan and one of his direct reports,',
 'finance chief Zinsner, said three people familiar with the change.  Since then, Intel Capital has invested in several companies in which Tan',
 'has a stake through his investment vehicles or venture capital firms, which include A&E Investment LLC, Celesta Capital and Walden',
 'International, three of the sources said. Some Intel staff have felt an obligation to explore such deals to win Tan’s support, two of them',
 'said.  One such investment was in proteanTecs, which announced a late-stage, Series D funding round in September. Intel Capital upped its',
 'existing stake in the startup, helping increase the value of Tan’s holdings through A&E Investment and Celesta Capital, funding data shows.',
 'ProteanTecs declined to comment. A&E Investment, Walden Catalyst and Walden International did not respond to requests for comment. In a',
 'statement, Celesta Capital said Tan "has always acted with integrity and a commitment to doing what is right for all stakeholders.”  Before',
 'Tan became CEO, Intel Capital co-invested with him or his investment firms at least 12 times since 2019, funding announcements show.  Intel',
 'is not alone in having a venture capital arm. That’s common at large technology companies, including Nvidia, Microsoft, Alphabet and',
 'Qualcomm. But unlike those other companies, Intel is unusual in having a CEO who oversees its venture unit while also leading unrelated',
 'investment firms, a Reuters review of leadership at those companies shows.  Reuters was unable to determine how much Intel’s recent',
 'investments had in total increased Tan’s net worth, estimated to be well over $500 million.  Two corporate governance experts consulted by',
 'Reuters said Tan’s dealmaking raises red flags due to the conflicts inherent in forging deals with his own portfolio companies.  One of them',
 "also said that Intel could benefit from Tan’s connections. “You don't want to preclude making good investments because your CEO is well",
 'connected,” said Daniel Taylor, a professor at the Wharton School specializing in corporate disclosures and insider trading.  TAN TOUTS HIS',
 'BOOK    The Intel CEO has consistently highlighted his portfolio in public appearances. In one of his first presentations as the Intel chief',
 'in March, he talked up his 251 chip-related investments, an online video from the occasion shows.   At an October chip conference in',
 'Phoenix, Tan gave a speech representing Intel that featured slides touting still more holdings, including in proteanTecs and SambaNova.',
 'Intel’s code of conduct encourages executives to disclose potential conflicts to the company’s board and top legal and compliance officers',
 'for resolution. “We avoid situations that interfere or appear to interfere with our ability to act in the best interests of Intel,” the code',
 'of conduct states. It adds that staff must mind conflicts arising from “an ownership interest in an Intel supplier, customer, or competitor”',
 'and from “outside employment that interferes with your obligations to Intel.”  The two corporate governance experts said Tan should have',
 'dropped his portfolio investments, placed them in a blind trust or set up a special committee of the board to remove potential conflicts',
 'with his investment portfolio. Some lawyers have said special committees are not always necessary, and some scholars argue outside board',
 'roles apprise executives of strategic information.  Intel declined to say if Tan had taken any such measure.  The Intel board’s independent',
 'Audit Committee “actively monitors, reviews, and approves, as appropriate, any related-party transactions in strict accordance with Intel’s',
 'rigorous Related-Party Transactions Policy,” the company spokesperson said.  The policy has exceptions. Transactions in which an Intel',
 'executive owns less than 10 per cent of a company that is party to a deal, whose value does not exceed $1 million or 2 per cent of that',
 'company’s revenue, are deemed to be “pre-cleared,” the policy states. It was not clear how many investments by Intel, if any, fell under',
 'this policy, which pre-dated Tan’s leadership.   INTEL EXPLORES BID FOR A STRUGGLING STARTUP  In 2018, SambaNova was a startup with lofty',
 'ambitions to build an AI computing system that could rival Nvidia’s AI hardware and software ecosystem. Tan’s venture firm Walden',
 'International co-led SambaNova’s $56 million Series A funding round that year, which secured Tan’s position on its board.   Over the years,',
 'SambaNova hoovered up money from investors: more from Tan’s venture firm and, around 2021, a big check from SoftBank, where Tan served as a',
 'board director until 2022. The $676 million Series D round led by SoftBank valued SambaNova at around $5 billion and gave Tan’s holdings a',
 'healthy increase on paper.  SoftBank declined to answer if Tan had a role in persuading it to invest in SambaNova.   But the startup’s',
 'ambitious vision wasn’t panning out, three people familiar with the matter said. Customers had more demand for Nvidia’s chips, which are',
 'good for a wide variety of uses in AI. That contrasts with the silicon from SambaNova, which is designed for more specific AI applications.',
 'In 2024, Tan stepped in as SambaNova’s new executive chairman in the hopes that he could help grow its business faster. The company was',
 'poised to run out of its cash, three people familiar with the matter said. It laid off 77 people in California, or reportedly about 15 per',
 'cent of its staff, in April.   The chip startup tried to drum up interest for another funding round, but found few takers, two sources said.',
 'Its revenue meant it would have to fundraise or go up for sale at a lower valuation, said the people and a third source. Bankers pegged its',
 'worth at $2 billion to around $3 billion at most, two of the sources said.  Tan asked Intel to look at a deal with the chipmaker over the',
 'summer, three sources said.   In recent weeks, some of SambaNova’s investors provided additional financing to tide over the startup, two',
 'people familiar with the insider round said.   In a statement to Reuters, a SambaNova spokesperson confirmed the startup recently secured',
 'additional funding, adding its business “is performing really well.”  Deal talks with SambaNova are ongoing, two of the sources said. Intel',
 'and SambaNova have signed a non-binding term sheet, one of them said.'''

In [8]:
annotations_to_make = [

    # -------- STARTUPS --------
    {"label": "Startup Name", "phrase": "Rivos"},
    {"label": "Startup Name", "phrase": "SambaNova"},
    {"label": "Startup Name", "phrase": "proteanTecs"},

    # -------- FOUNDERS / EXECUTIVES --------
    {"label": "Founder Name", "phrase": "Lip-Bu Tan"},

    # -------- BUYERS / ACQUIRERS --------
    {"label": "Company Buying Startup", "phrase": "Intel"},
    {"label": "Company Buying Startup", "phrase": "Meta"},

    # -------- INVESTMENT FIRMS --------
    {"label": "Investment Company", "phrase": "Walden Catalyst"},
    {"label": "Investment Company", "phrase": "Intel Capital"},
    {"label": "Investment Company", "phrase": "SoftBank"},
    {"label": "Investment Company", "phrase": "Nvidia"},
    {"label": "Investment Company", "phrase": "Walden International"},
    {"label": "Investment Company", "phrase": "Celesta Capital"},
    {"label": "Investment Company", "phrase": "A&E Investment LLC"},

    # -------- FUNDING AMOUNTS --------
    {"label": "Funding Amount", "phrase": "$2 billion"},
    {"label": "Funding Amount", "phrase": "$4 billion"},
    {"label": "Funding Amount", "phrase": "$5 billion"},
    {"label": "Funding Amount", "phrase": "$8.9 billion"},
    {"label": "Funding Amount", "phrase": "$56 million"},
    {"label": "Funding Amount", "phrase": "$676 million"}
]


In [14]:

# -------------------------------------------
# CONFIG: Define your label schema here
# -------------------------------------------
LABEL_MAP = {
    'Startup Name': 'STARTUP',
    'Founder Name': 'FOUNDER',
    'Investment Company': 'INVESTOR',
    'Funding Amount': 'INVESTMENT',
    'Revenue': 'ARR',
    'Percent change in revenue': 'PERCHNG',
    'Company Buying Startup': 'BUY',
    "Valuation": "VAL"
}

In [9]:
# -------------------------------------------
# FUNCTION: Find all occurrences of a substring
# Returns start, end, and text
# -------------------------------------------
def find_all_offsets(text, phrase):
    pattern = re.escape(phrase)
    matches = list(re.finditer(pattern, text))

    entities = []
    for m in matches:
        start = m.start()
        end = m.end()
        entities.append({
            "start": start,
            "end": end,
            "text": text[start:end]
        })

    return entities

In [10]:
# -------------------------------------------
def annotate_article(article_text, annotations):
    output = []

    for item in annotations:
        human_label = item["label"]              # e.g. "Startup Name"
        phrase = item["phrase"]                  # e.g. "Rivos"
        machine_label = LABEL_MAP[human_label]   # e.g. "STARTUP"

        offsets = find_all_offsets(article_text, phrase)

        if len(offsets) == 0:
            print(f"[WARNING] Phrase not found: {phrase}")

        output.append({
            "label": [human_label],
            "points": offsets
        })

    return output

In [15]:
text2 = df.loc[6, "content"]
print(text2)

While Google and OpenAI battle for model dominance, Anthropic is quietly winning the enterprise AI race
Follow ZDNET: Add us as a preferred source on Google.

 ZDNET's key takeaways 

Anthropic has toppled OpenAI in share of enterprise GenAI spending.Enterprise spending represents an AI boom, not a bubble.Agentic AI is still a niche compared to prompt engineering.

The most popular single use of generative artificial intelligence in the enterprise is coding tools for programmers. That tool's popularity has helped Anthropic trounce OpenAI in the enterprise in 2025, according to a new survey of company spending. 

"The foundation model landscape shifted decisively this year when Anthropic surprised industry watchers by unseating OpenAI as the enterprise leader," according to the report, the third annual State of Generative AI in the enterprise, from venture capital firm Menlo Ventures. 

Also: How Anthropic's enterprise dominance fueled its monster $183B valuation

The report estimated t

In [18]:
import re
# 3. Get annotations
annotations = annotate_article(article_text, annotations_to_make)

In [19]:
annotations_to_make2 = [

    # -------- STARTUPS / AI COMPANIES --------
    {"label": "Startup Name", "phrase": "Anthropic"},
    {"label": "Startup Name", "phrase": "Replit"},
    {"label": "Startup Name", "phrase": "Cursor"},
    {"label": "Startup Name", "phrase": "Harness"},
    {"label": "Startup Name", "phrase": "Windsurf"},
    {"label": "Startup Name", "phrase": "Augment Code"},
    {"label": "Startup Name", "phrase": "All Hands AI"},
    {"label": "Startup Name", "phrase": "Writer"},
    {"label": "Startup Name", "phrase": "Glean"},

    # -------- INVESTMENT / BIG TECH / VC FIRMS --------
    {"label": "Investment Company", "phrase": "Menlo Ventures"},
    {"label": "Investment Company", "phrase": "Google"},
    {"label": "Investment Company", "phrase": "Amazon"},
    {"label": "Investment Company", "phrase": "Microsoft"},
    {"label": "Investment Company", "phrase": "Salesforce"},

    # -------- FOUNDERS / AUTHORS / EXECUTIVES --------

    {"label": "Founder Name", "phrase": "Jeremy Burton"},

    # -------- FUNDING / MARKET SIZE / SPEND --------


    #---------------- VALUATION ------------------------
    {"label": "Valuation", "phrase": "$183B"},
    {"label": "Valuation", "phrase": "$4 billion"},

    #------------- REVENUE ----------------------------
    {"label": "Revenue", "phrase": "$37 billion"},
    {"label": "Revenue", "phrase": "$18 billion"},
    {"label": "Revenue", "phrase": "$8.4 billion"},
    {"label": "Revenue", "phrase": "$4.2 billion"},
    {"label": "Revenue", "phrase": "$288 billion"},
    {"label": "Revenue", "phrase": "$9.5 billion"},


    # -------- PERCENT CHANGE / MARKET SHARE --------
    {"label": "Percent change in revenue", "phrase": "24%"},
    {"label": "Percent change in revenue", "phrase": "12%"},
    {"label": "Percent change in revenue", "phrase": "50%"},
    {"label": "Percent change in revenue", "phrase": "16%"}
]


In [20]:
annotations2 = annotate_article(text2, annotations_to_make2)

In [21]:
annotations2

[{'label': ['Startup Name'],
  'points': [{'start': 52, 'end': 61, 'text': 'Anthropic'},
   {'start': 184, 'end': 193, 'text': 'Anthropic'},
   {'start': 519, 'end': 528, 'text': 'Anthropic'},
   {'start': 686, 'end': 695, 'text': 'Anthropic'},
   {'start': 909, 'end': 918, 'text': 'Anthropic'},
   {'start': 1004, 'end': 1013, 'text': 'Anthropic'},
   {'start': 1833, 'end': 1842, 'text': 'Anthropic'},
   {'start': 1947, 'end': 1956, 'text': 'Anthropic'},
   {'start': 2081, 'end': 2090, 'text': 'Anthropic'},
   {'start': 2280, 'end': 2289, 'text': 'Anthropic'},
   {'start': 2671, 'end': 2680, 'text': 'Anthropic'},
   {'start': 3038, 'end': 3047, 'text': 'Anthropic'},
   {'start': 3105, 'end': 3114, 'text': 'Anthropic'},
   {'start': 3247, 'end': 3256, 'text': 'Anthropic'}]},
 {'label': ['Startup Name'],
  'points': [{'start': 2417, 'end': 2423, 'text': 'Replit'},
   {'start': 2917, 'end': 2923, 'text': 'Replit'},
   {'start': 3020, 'end': 3026, 'text': 'Replit'}]},
 {'label': ['Startup 

In [22]:

text3 = df.loc[8, "content"]

In [ ]:
annotated_dict = {"new_article": [article_text, text2, text3], "annotations": [annotations, annotations2]}

In [24]:
df.content

0    Intel pursued deals that boosted CEO Lip-Bu Ta...
1    We’re not late to qcomm: Amazon's Amit Agarwal...
2    Amazon's $35 billion India top-up; Meesho inve...
3    Entrepreneurs Don&#39;t Have Time to Read  and...
4    Only Hours Left to Save Big on this AI-Powered...
5    Microsoft’s India power play; Insurers’ Fastag...
6    While Google and OpenAI battle for model domin...
7    Swiggy’s Rs 10,000 crore QIP; PhysicsWallah’s ...
8    Bengaluru-based seafood supply chain startup ‘...
9    Gaming CEO says Cyprus is ‘rocket fuel’ for gl...
Name: content, dtype: object

In [25]:
annotated_dict

{'new_article': ['Intel pursued deals that boosted CEO Lip-Bu Tan&#39;s fortune, sources say SAN FRANCISCO, Dec 10 : When the chairman of AI chip startup\',\n \'Rivos wanted Intel to bid for the company, he had no need to phone the chip giant. That’s because the chairman of Rivos was also Intel’s\',\n \'CEO: Lip-Bu Tan.  Tan had pitched Intel’s board on buying Rivos in the summer of 2025, but he had no luck. The board told Tan he had a\',\n \'conflict in representing both Rivos’ interests and Intel’s, and he lacked a strategy on artificial intelligence to justify a deal, three\',\n \'people familiar with the events told Reuters.  Tan asked one of his lieutenants at Intel to pitch a new AI plan, leading to partnership\',\n \'talks with Rivos, the people said. But now there was a problem: social media giant Meta had been stalking Rivos and made an offer for the\',\n \'company.  Meta’s interest spurred Intel to make its own offer. Meta countered with a sweetened bid. The competition for t

In [1]:
import re
import pickle
import pandas as pd
with open("../artifacts/annotated_data_dict.pkl", "rb") as f:
    annotated_data_dict = pickle.load(f)

df = pd.DataFrame(annotated_data_dict)
print(df.shape)
df.head()

(10, 2)


,news_articles,annotations
0,Harness hits $5.5B valuation with $240M raise ...,"[{'label': ['Startup Name'], 'points': [{'star..."
1,Intel pursued deals that boosted CEO Lip-Bu Ta...,"[{'label': ['Startup Name'], 'points': [{'star..."
2,Exploring the role of quantum computing in nex...,"[{'label': ['Startup Name'], 'points': [{'star..."
3,Israeli startup Port raises $100m at $800m val...,"[{'label': ['Startup Name'], 'points': [{'star..."
4,"Taxed, Unregulated & Still Growing: Inside Ind...","[{'label': ['Startup Name'], 'points': [{'star..."


In [5]:
df.to_csv("../artifacts/annotated_data.csv", index=False, header=True)

In [14]:
for annot in df["annotations"][0]:
    print(annot["label"][0])
    print("\n")
    for t in annot["points"]:
        start = t["start"]
        end = t["end"]
        text = t["text"]
        if text != df["news_articles"][0][start:end]:
            matches = re.finditer(text, df["news_articles"][0])
            for m in matches:
                print("Regex match - ", df["news_articles"][0][m.start():m.end()])
                print(text)
    print("\n")

Startup Name


Regex match -  AppDynamics
AppDynamics
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  Harness
Harness
Regex match -  H

In [16]:
annotated_data_dict["news_articles"][0]

'Harness hits $5.5B valuation with $240M raise to automate AI’s ‘after-code’ gap\nAI DevOps tool Harness, founded in 2017 by serial entrepreneur Jyoti Bansal, is on track to exceed $250 million in annual recurring revenue in 2025, Bansal tells TechCrunch. \n\nThe startup just raised a fresh $240 million Series E funding round that values the company at $5.5 billion post-money.\n\nThe round includes a $200 million primary investment led by Goldman Sachs and a planned $40 million tender offer with participation from IVP, Menlo Ventures, and Unusual Ventures. The tender offer is intended to provide some liquidity to its long-term employees, Bansal said.\n\nThe new valuation is a 49% jump from its $3.7 billion valuation in a $230 million round in April 2022. With this funding, the startup has raised $570 million of equity to date.\n\nAs AI accelerates code production, it is widening a bottleneck in the far larger “after-code” phase of software development — the testing, security checks, an

In [33]:
count = {}
for i in range(len(annotated_data_dict["annotations"][0])):
    annot = annotated_data_dict["annotations"][0][i]
    print(annot["label"][0])
    for j in range(len(annot["points"])):
        t = annot["points"][j]
        start = t["start"]
        end = t["end"]
        text = t["text"]

        if text in list(count.keys()):
            count[text] += 1
        else:
            count[text] = 1

        if text != annotated_data_dict["news_articles"][0][start:end]:
            matches = re.finditer(text, annotated_data_dict["news_articles"][0])
            if count[text]-1<len(list(matches)):
                match = list(matches)[count[text]-1]
                print("Text = ", text)
                print("Regex match = ", match.group())
                if text == match.group():
                    annotated_data_dict["annotations"][0][i]["points"][j]["start"] = match.start()
                    annotated_data_dict["annotations"][0][i]["points"][j]["end"] = match.end()


            

Startup Name
Founder Name
Valuation
Funding Amount
Revenue
Investment Company


IndexError: list index out of range

In [23]:
count = {}
count["word"] = 1
count["word"] += 1

In [25]:
list(count.keys())

['word']

In [92]:
def correct_annotations(row):
    article_text = row["news_articles"]
    raw_annotations = row["annotations"]
    
    # Flatten the raw_annotations json
    all_ent = []
    for entry in raw_annotations:
        label_name = entry.get("label")[0]
        points = entry.get("points")
        for point in points:
            all_ent.append({
                "label": label_name,
                "text": point["text"],
                "orig_start": point["start"]
            })
    all_ent.sort(key=lambda x: x["orig_start"])
    # Keeps track of the end of a unique text
    search_cursors = {}
    corrected_points = []
    for ent in all_ent:
        text_to_find = ent["text"]
        label = ent["label"]
        # Index where to start the search from, 0 if not already searched
        start_search_idx = search_cursors.get(text_to_find, 0)
        # Start index of the text to find
        found_start = article_text.find(text_to_find, start_search_idx)
        if found_start != -1:
            found_end = found_start + len(text_to_find)
            search_cursors[text_to_find] = found_end
            corrected_points.append({
                "label": label,
                "start": found_start,
                "end": found_end,
                "text": text_to_find
            })
        else:
            print(f"Could not find {text_to_find} after {start_search_idx}")
        
    grouped_annotations = {}
    for point in corrected_points:
        lbl = point["label"]
        if lbl not in grouped_annotations:
            grouped_annotations[lbl] = []
        grouped_annotations[lbl].append({
            "start": point["start"],
            "end": point["end"],
            "text": point["text"]
        })
    
    final_output = []
    for lbl_name, points in grouped_annotations.items():
        points.sort(key=lambda x: x["start"])
        final_output.append({
            "label": [lbl_name],
            "points": points
        })
    
    return final_output



In [ ]:
def verify_annotations(annot: list, article_text: str):
    for entry in annot:
        for point in entry["points"]:
            text_to_verify = point["text"]
            start = point["start"]
            end = point["end"]
            actual_text = article_text[start:end]
            if text_to_verify != actual_text:
                print(f"Actual text = {actual_text}")
                print(f"Text = {text_to_verify}")
        

In [98]:
df["correct_annotations"] = df.apply(correct_annotations, axis=1)

Could not find A&E Investment LLC after 7827


In [102]:
df

,news_articles,annotations,correct_annotations
0,Harness hits $5.5B valuation with $240M raise ...,"[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Startup Name'], 'points': [{'star..."
1,Intel pursued deals that boosted CEO Lip-Bu Ta...,"[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Investment Company'], 'points': [..."
2,Exploring the role of quantum computing in nex...,"[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Funding Amount'], 'points': [{'st..."
3,Israeli startup Port raises $100m at $800m val...,"[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Startup Name'], 'points': [{'star..."
4,"Taxed, Unregulated & Still Growing: Inside Ind...","[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Funding Amount'], 'points': [{'st..."
5,Amazon's $35 billion India top-up; Meesho inve...,"[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Funding Amount'], 'points': [{'st..."
6,"Here’s the pitch deck used by Yoodli, the Seat...","[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Startup Name'], 'points': [{'star..."
7,Lovable becomes a unicorn with $200M Series A ...,"[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Startup Name'], 'points': [{'star..."
8,AI inference startup Runware raises $50M to ma...,"[{'label': ['Startup Name'], 'points': [{'star...","[{'label': ['Startup Name'], 'points': [{'star..."
9,Meta to pay nearly $15 billion for Scale AI st...,"[{'label': ['Company Buying Startup'], 'points...","[{'label': ['Company Buying Startup'], 'points..."


In [105]:
for idx, row in df.iterrows(): 
    verify_annotations(row["correct_annotations"], row["news_articles"])

 - All annotations corrected and verified
 - Saving updated annotations dataframe as dict in pkl file

In [106]:
updated_annotations_dict = df.to_dict()

 - Saving as pkl file

In [110]:
import pickle
with open("../artifacts/annotated_data_dict.pkl", "wb") as f:
    pickle.dump(updated_annotations_dict, f)

In [10]:
tag_vals = set(["X", "[CLS]", "[SEP]"])
tag_vals

{'X', '[CLS]', '[SEP]'}

In [11]:
tag_vals = {'B-INVESTMENT', 'X', 'L-PERCHNG', 'O', 'L-INVESTOR', 'B-ARR', 'B-INVESTOR', 'CLS', 'I-ARR', 'L-VAL', 'U-STARTUP', 'B-STARTUP', 'I-INVESTMENT', 'U-INVESTOR', 'SEP', 'L-STARTUP', 'B-FOUNDER', 'L-FOUNDER', 'I-VAL', 'L-ARR', 'U-BUY', 'L-INVESTMENT', 'B-PERCHNG', 'B-VAL'}

In [12]:
tag_vals

{'B-ARR',
 'B-FOUNDER',
 'B-INVESTMENT',
 'B-INVESTOR',
 'B-PERCHNG',
 'B-STARTUP',
 'B-VAL',
 'CLS',
 'I-ARR',
 'I-INVESTMENT',
 'I-VAL',
 'L-ARR',
 'L-FOUNDER',
 'L-INVESTMENT',
 'L-INVESTOR',
 'L-PERCHNG',
 'L-STARTUP',
 'L-VAL',
 'O',
 'SEP',
 'U-BUY',
 'U-INVESTOR',
 'U-STARTUP',
 'X'}

In [13]:
tag2idx = {i: t for i, t in enumerate(tag_vals)}

In [14]:
tag2idx

{0: 'O',
 1: 'U-BUY',
 2: 'I-VAL',
 3: 'CLS',
 4: 'L-STARTUP',
 5: 'U-INVESTOR',
 6: 'U-STARTUP',
 7: 'B-FOUNDER',
 8: 'L-ARR',
 9: 'L-PERCHNG',
 10: 'I-ARR',
 11: 'B-ARR',
 12: 'L-FOUNDER',
 13: 'L-INVESTMENT',
 14: 'I-INVESTMENT',
 15: 'SEP',
 16: 'B-INVESTMENT',
 17: 'B-PERCHNG',
 18: 'B-VAL',
 19: 'X',
 20: 'L-INVESTOR',
 21: 'B-INVESTOR',
 22: 'L-VAL',
 23: 'B-STARTUP'}

SyntaxError: invalid decimal literal (268327555.py, line 1)

In [4]:
nd = [1,2,3,5,6,7]
n = [9,9,10]
any(i in n for i in nd)

False